### GridSearchCV

- 모델의 일반화 성능을 개선하기 위한 매개변수 튜닝
- GridSearchCV(estimator, param_grid)
    - 조정하고자 하는 매개변수명과 설정값에 대해서 딕셔너리 타입으로 전달
    - 기본값으로 3-fold cross-validation을 적용
    - 분석모델로 분류기가 전달되면 stratified 3-fold cross-validation 적용
    - 일반적인 분석 모델 객체와 유사하게 GridSearch 객체도 predict, score 메서드 제공
        - predict() : 찾아낸 최적의 매개변수로 학습된 모델로부터 예측값 얻기
        - score() : 찾아낸 최적의 매개변수로로 학습된 모델의 일반화 성능 평가
    - 최적의 매개변수를 찾는 과정에서는 학습 데이터만 사용할 뿐 테스트 데이터는 사용하지 않음
        - grid_scores_
            - 모든 하이퍼파리미터 조합에 대한 성능 결과
            - parameters: 사용된 파라미터
            - mean_validation_score: 교차 검증(cross-validation) 결과의 평균값
            - cv_validation_scores: 모든 교차 검증(cross-validation) 결과
        - best_params_ : 찾아낸 최적의 매개변수
        - best_score_ : 최적의 매개변수로 학습 데이터에서 얻은 최상의 교차 검증 정확도
        - best_estimator_ : 최적의 매개변수에서 학습 데이터를 사용해서 학습한 모델 객체

In [2]:
# IRIS 데이터셋 사용
from sklearn import datasets

In [3]:
iris = datasets.load_iris()

In [4]:
# 독립변수, 종속변수 분리
iris_x = iris.data
iris_y = iris.target

In [6]:
# 학습, 평가 데이터 분리 (7:3)
from sklearn.model_selection import train_test_split

In [7]:
train_x, test_x, train_y, test_y = train_test_split(iris_x, iris_y, test_size=.3)

In [8]:
train_x.shape

(105, 4)

In [11]:
test_x.shape

(45, 4)

In [12]:
# RandomForest 모델 객체 생성
from sklearn.ensemble import RandomForestClassifier

In [13]:
rf = RandomForestClassifier()

In [14]:
# 그리드서치 라이브러리
from sklearn.model_selection import GridSearchCV

In [18]:
# GridSearchCV(model, params)
# 그리드서치를 수행할 하이퍼파라미터 맵 생성
# n_estimator : 단일 분류기 개수
# max_features : 독립변수 최대 개수
# max_depth : 가치지기 최대 깊이
# criterion : 불순도 지표
param_map = {'n_estimators' : [100,200],
             'max_depth' : [5,10]}

In [19]:
# 하이퍼파라미터 맵과 모델을 결합
grid_search = GridSearchCV(estimator=rf, param_grid=param_map)

In [20]:
# 전달한 모든 경우에 수에 따라 모델을 학습
grid_search.fit(train_x, train_y)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100, 200], 'max_depth': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
# 학습결과를 기반으로 최고의 성능을 가지는 하이퍼파라미터 조합 반환
grid_search.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [22]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
grid_search.best_score_

0.9238095238095239

In [24]:
# best_params_ 조합으로 모델 예측 수행
grid_search.score(test_x, test_y)

0.9777777777777777

In [25]:
best_rf = grid_search.best_estimator_

In [26]:
best_rf.score(test_x, test_y)

0.9777777777777777

### <실습문제> XGBoost 모델에 대한 GridSearch 적용

- make_moons 데이터셋을 이용
    - 샘플개수 : 10000
    - 표준편차 : 0.25
    
- 학습, 평가 데이터 비율 : 8 대 2
- 파라미터 맵 
    - learning_rate : 0.01, 0.1, 0.2
    - max_depth : 4,6,8
    - n_estimators : 100, 300, 500

In [27]:
# 데이터셋 생성
from sklearn.datasets import make_moons

x, y = make_moons(n_samples=10000, noise=.25)

In [28]:
# 학습, 평가 데이터 분리
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.2)

In [29]:
from xgboost import XGBClassifier

xg1 = XGBClassifier()

In [30]:
xg1.fit(train_x, train_y)
xg1.score(test_x, test_y)

0.9435

In [ ]:
# 기본 파라미터를 사용한 XGBoost score : 0.9435

In [31]:
# 파라미터 맵 생성
param_map = {'learning_rate':[0.01, 0.1, 0.2],
            'max_depth' : [4,6,8],
            'n_estimators' :[100,300,500]}

In [32]:
xg2 = XGBClassifier()

In [33]:
# GridSearchCV 객체 생성
grid = GridSearchCV(xg2, param_grid=param_map)

In [35]:
grid.fit(train_x, train_y)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [4, 6, 8], 'n_estimators': [100, 300, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [36]:
# 최적의 파라미터 조합 반환
grid.best_params_

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300}

In [37]:
# 최적 파라미터에서 일반화 성능 확인 : 0.9445
grid.score(test_x, test_y)

0.9445